In [1]:
from preprocessing import ImdbDataset
import pandas as pd

In [7]:
imdb = ImdbDataset('data/aclImdb_v1.tar.gz')

In [14]:
tr = {'train_sentence': imdb.train_sentences, 'train_label': imdb.train_labels}
ts= {'test_sentence': imdb.test_sentences, 'test_label': imdb.test_labels}
df_tr = pd.DataFrame(data=tr)
df_ts = pd.DataFrame(data=ts)
df_tr

,train_sentence,train_label
0,There are many police dramas doing the rounds....,1
1,"Chilling, majestic piece of cinematic fright, ...",1
2,I'd just like to say that i've seen this film ...,1
3,This 1981 comedy still sparkles thanks to the ...,1
4,"One doesn't get to enjoy this gem, the 1936 In...",1
...,...,...
24995,Pretty awful but watchable and entertaining. I...,0
24996,Oh man is this movie bad. It flows horribly. T...,0
24997,Joyce Reynolds seems a might grown-up for the ...,0
24998,"I liked Boyle's performance, but that's about ...",0


In [15]:
len(imdb.test_sentences)

25000

In [16]:
import torchtext

In [23]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english', )
texts = [
    'I love my dog!!',
    'I love my cat',
    'You love my dog!',
    'Do you think my dog is amazing?'
]
tokens = [tokenizer(text) for text in texts]
print(tokens)

vocab = torchtext.vocab.Vocab(specials=['<unk>', '<pad>', '<bos>', '<eos>'])
vocab.build_vocab_from_iterator(tokens)
word_index = vocab.stoi
print(word_index)

[['i', 'love', 'my', 'dog', '!', '!'], ['i', 'love', 'my', 'cat'], ['you', 'love', 'my', 'dog', '!'], ['do', 'you', 'think', 'my', 'dog', 'is', 'amazing', '?']]


TypeError: Vocab.__init__() got an unexpected keyword argument 'specials'

In [1]:
# Define the input sentence
input_sentence = "This is an example sentence for testing purposes"

# Apply the SentencePiece tokenizer
tokenizer = T.SentencePieceTokenizer(xlmr_spm_model_path)
tokens = tokenizer(input_sentence)
print("Tokens: ", tokens)
# Output: Tokens: ['▁This', '▁is', '▁an', '▁example', '▁sentence', '▁for', '▁testing', '▁purposes']

# Apply the VocabTransform
vocab_transform = T.VocabTransform(load_state_dict_from_url(xlmr_vocab_path))
indexed_tokens = vocab_transform(tokens)
print("Indexed tokens: ", indexed_tokens)
# Output: Indexed tokens: [238, 13, 62, 207, 2559, 29, 4061, 1331]

# Truncate the sequence to a maximum length of 254 (to account for <bos> and <eos> tokens)
truncate_transform = T.Truncate(254)
truncated_tokens = truncate_transform(indexed_tokens)
print("Truncated tokens: ", truncated_tokens)
# Output: Truncated tokens: [238, 13, 62, 207, 2559, 29, 4061, 1331]

# Add <bos> and <eos> tokens to the sequence
add_token_transform1 = T.AddToken(bos_idx, begin=True)
add_token_transform2 = T.AddToken(eos_idx, begin=False)
bos_eos_tokens = add_token_transform2(add_token_transform1(truncated_tokens))
print("Tokens with <bos> and <eos>: ", bos_eos_tokens)
# Output: Tokens with <bos> and <eos>: [0, 238, 13, 62, 207, 2559, 29, 4061, 1331, 2]


NameError: name 'T' is not defined

In [18]:
from collections import Counter


In [40]:
word_counts = Counter()
texts = [
    'I love my dog!!',
    'I love my cat',
    'You love my dog!',
    'Do you think my dog is amazing?',
    'my house is amazing'
]
for text in texts:
    words = text.split()
    word_counts.update(words)
most_common = word_counts.most_common(5)
for idx, (word, count) in enumerate(most_common):
    print(f"idx: {idx}\t word: {word}\t count: {count}")
word2index = {}
word2index = {word: idx + 2 for idx, (word, count) in enumerate(most_common)}
word2index['<OOV>'] = 1
word2index['<PAD>'] = 0
word2index


idx: 0	 word: my	 count: 5
idx: 1	 word: love	 count: 3
idx: 2	 word: I	 count: 2
idx: 3	 word: is	 count: 2
idx: 4	 word: dog!!	 count: 1


{'my': 2, 'love': 3, 'I': 4, 'is': 5, 'dog!!': 6, '<OOV>': 1, '<PAD>': 0}

In [41]:
for word, idx, in word2index.items():
    print(f"idx: {idx}, word: {word}")

idx2word = {idx: word for word, idx, in word2index.items()}
idx2word

idx: 2, word: my
idx: 3, word: love
idx: 4, word: I
idx: 5, word: is
idx: 6, word: dog!!
idx: 1, word: <OOV>
idx: 0, word: <PAD>


{2: 'my', 3: 'love', 4: 'I', 5: 'is', 6: 'dog!!', 1: '<OOV>', 0: '<PAD>'}

In [50]:
sequences = []
for text in texts: 
    words = text.split()
    sequence = [word2index.get(word, 1) for word in words]
    sequences.append(sequence)
sequences

[[4, 3, 2, 6], [4, 3, 2, 1], [1, 3, 2, 1], [1, 1, 1, 2, 1, 5, 1], [2, 1, 5, 1]]

In [98]:
class Tokenizer():
    def __init__(self, num_words, oov_token=None, pad_token=None):
        self.num_words = num_words
        self.oov_token = oov_token
        self.pad_token = pad_token

    def fit_on_texts(self, texts):
        word_counts = Counter()
        for text in texts:
            words = text.split()
            word_counts.update(words)

        most_common = word_counts.most_common(self.num_words)
        first_tokens = [self.oov_token,self.pad_token]
        first_tokens_nn = list(filter(lambda s:s is not None, first_tokens))
        lens = len(first_tokens_nn)
        self.word_index = {word: (idx+1) + lens for idx, (word, count) in enumerate(most_common)}

        if lens > 0:
            list_len = [i+1 for i in range(len(first_tokens_nn))]

            if self.pad_token is not None:
                self.word_index[self.pad_token] = list_len[0]
            if self.oov_token is not None:
                self.word_index[self.oov_token] = list_len[-1]
            
        self.index_word = {index: word for word, index in self.word_index.items()}

    def texts_to_sequences(self, texts):
        sequences = []
        for text in texts:
            words = text.split()
            sequence = [self.word_index.get(word, 1) for word in words]
            sequences.append(sequence)
        return sequences



texts = [
    'I love my dog',
    'I love my cat',
    'You love my dog!',
    'Do you think my dog is amazing?',
    'my house is amazing'
]

tokenizer = Tokenizer(num_words=6, oov_token='<oov>')

tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index

print(word_index)
index_word = tokenizer.index_word
index_word


{'my': 2, 'love': 3, 'I': 4, 'dog': 5, 'is': 6, 'cat': 7, '<oov>': 1}


{2: 'my', 3: 'love', 4: 'I', 5: 'dog', 6: 'is', 7: 'cat', 1: '<oov>'}

In [83]:
lists = ['one', ]
lens = [i+1 for i in range(len(lists))]
lens[1]

IndexError: list index out of range

In [20]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
sp = """
"""
test_sentence = """When forty winters shall besiege thy brow
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [22]:
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]

print(ngrams[:3])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [27]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [ ]:
from torch import nn

In [ ]:
class NGramLM(nn.Module):
    def __init__(self, vocab_size, ):
        super().__init__()

In [ ]:
train, test, val = imdb.create_dataset(split=0.2)
print

In [ ]:
def create_dataset(self, val_split = None):
        self.extract()
        corpi = ['train', 'test']
        train = []
        test = []
        val =  []
        for corpus in corpi:
            path = os.path.join(self.dataset_dir, corpus)
            if corpus == 'train':
                pos_sentences, pos_labels = self.append(path, 'pos', 1)
                neg_sentences, neg_labels = self.append(path, 'neg', 0)
                sentences = pos_sentences + neg_sentences
                labels = pos_labels + neg_labels
                train.append([sentences, labels])
            elif corpus == 'test':
                pos_sentences, pos_labels = self.append(path, 'pos', 1)
                neg_sentences, neg_labels = self.append(path, 'neg', 0)
                sentences = pos_sentences + neg_sentences
                labels = pos_labels + neg_labels
                test.append([sentences, labels])